In [2]:
'./Notebooks/data.yaml'
import json
import os
from ultralytics import YOLO
import shutil
import yaml  # for importing a yaml file
import joblib
import torch
import subprocess
from io import BytesIO
import tarfile
import pandas as pd

In [3]:
current_directory = os.getcwd()
last_folder = os.path.basename(current_directory)
    
if last_folder != "project-danielteresa":
    while last_folder != "project-danielteresa":
        parent_directory = os.path.dirname(current_directory)
        last_folder = os.path.basename(parent_directory)

        os.chdir(parent_directory)
        print(f"Changed directory to: {parent_directory}")
else:
    print("Already in the project root directory.")

# our modules
from src.mymodule import * # for importing our functions

Changed directory to: C:\Users\daniq\PycharmProjects\project-danielteresa


In [4]:
# Load the YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Function to set variables globally and store their names
def set_variables(config, prefix='', var_dict={}):
    for key, value in config.items():
        if isinstance(value, dict):
            set_variables(value, prefix + key + '_', var_dict)
        else:
            globals()[prefix + key] = value
            var_dict[prefix + key] = value
    return var_dict

# Set variables globally and get a dictionary of the set variables
set_vars = set_variables(config)

# Print all the variables that were set
print("Variables set from YAML file:")
for var_name, var_value in set_vars.items():
    print(f"{var_name}: {var_value}")

Variables set from YAML file:
seed: 123
color1: #62b6cb
color2: #fb8500
color3: #023047
color4: #FFB703
path_annotations: ./Data
path_train: ./Data/train/original
path_train_train: ./Data/train/train
path_train_aug: ./Data/train/train_aug
path_train_val: ./Data/train/val
path_test: ./Data/test
path_yolo: ./Data/Yoloimages/
path_models: ./Models
kaggle_train_annotations: annotations_train.json
train_annotations_name_temp: annotations_train_temp.json
train_annotations_name: annotations_train_updated.json
aug_train_annotations_name: annotations_train_updated_aug.json
val_annotations_name: annotations_val_updated.json
kaggle_test_annotations: annotations_test.json
test_annotations_name_temp: annotations_test_temp.json
test_annotations_name: annotations_test_updated.json
weights_yolo_path: Models/yolo_weights
runs_path: Models/runs
bucket_name: sagemaker-eu-west-1-project-danielteresa/
bucket_name2: sagemaker-eu-west-1-project-danielteresa
tuning_jobname: YOLO-tuning8
final_jobname: YOLO-fi

In [5]:
 yolo_model = YOLO("Models/best_modeltuning_YOLO-tuning7/train/weights/best.pt")

In [36]:
results = yolo_model.predict("API/inputs/mix1.jpg",conf=0.15)


image 1/1 C:\Users\daniq\PycharmProjects\project-danielteresa\API\inputs\mix1.jpg: 384x640 1 mis_lost, 1 glass_crack, 286.5ms
Speed: 0.0ms preprocess, 286.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


In [37]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'mis_lost', 1: 'met_tear', 2: 'met_scratch', 3: 'glass_crack', 4: 'mis_punct', 5: 'mis_lamp', 6: 'met_dent_minor', 7: 'met_dent_medium', 8: 'met_dent_severe'}
 obb: None
 orig_img: array([[[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   1],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],
 
        [[250, 252, 252],
         [251, 253, 253],
         [249, 253, 254],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],
 
        [[251, 255, 255],
         [248, 255, 255],
         [244, 255, 255],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],
 
        ...,
 
        [[224, 229, 232],
         [227, 232, 235],
         [227, 232, 235],
         ...,
       

In [46]:
#print(dir(results[0].masks))
#results[0].probs
results[0].names

{0: 'mis_lost',
 1: 'met_tear',
 2: 'met_scratch',
 3: 'glass_crack',
 4: 'mis_punct',
 5: 'mis_lamp',
 6: 'met_dent_minor',
 7: 'met_dent_medium',
 8: 'met_dent_severe'}

In [45]:
results[0].boxes.cpu().numpy().cls

array([          3,           0], dtype=float32)

In [39]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen

In [41]:
len(results)

1

We upload the cost estimation model

In [57]:
cost_model=joblib.load("Models/cost_model.pkl")

'brand': array(['Toyota', 'Volkswagen'], dtype=object),

 'model': array(['Corolla', 'C-HR', 'Yaris', 'Golf', 'Polo', 'Tiguan'], dtype=object),
 
 'veh_age_range': array(['Old', 'Newer', 'Middle'], dtype=object),
 
 'workshop_quality': array(['High', 'Low', 'Medium'], dtype=object),
 
 'counties': array(['Cork', 'Dublin', 'Tipperary', 'Meath', 'Clare', 'Kildare', 'Wexford', 'Donegal', 'Galway', 'Carlow', 'Roscommon', 'Waterford', 'Westmeath', 'Leitrim', 'Kilkenny', 'Cavan', 'Limerick', 'Longford', 'Kerry', 'Wicklow', 'Mayo', 'Sligo', 'Offaly', 'Laois', 'Louth', 'Monaghan'], dtype=object),
 
 'damage_type': array(['mis_lost', 'met_tear', 'mis_lamp', 'glass_crack', 'mis_punct', 'met_dent_minor', 'met_dent_medium', 'met_dent_severe'], dtype=object)
 

group1 = ["Kildare", "Dublin"]


group2 = ["Meath", "Cork", "Westmeath", "Laois", "Galway", "Kilkenny", "Limerick", "Clare"]

In [66]:
workshops={
'Smithfield Autotech':'Low',
'Sandyford McCann Motors':'Medium',
'Mobile Mechanic':'High'
}

selected_workshop='Sandyford McCann Motors'

selected_workshop_qual= workshops[selected_workshop]

'Medium'

In [68]:
case = pd.DataFrame({
'const': [1], 
'brand_Volkswagen': [0], 
'model_Corolla': [0], 
'model_Golf': [0],
'model_Polo': [0], 
'model_Tiguan': [0], 
'model_Yaris': [1], 
'veh_age_range_Newer': [1],
'veh_age_range_Old': [0], 
'workshop_quality_Low': [0], 
'workshop_quality_Medium': [0],
'counties_group2': [0], 
'counties_group3': [0], 
'damage_type_met_dent_medium': [0],
'damage_type_met_dent_minor': [0], 
'damage_type_met_dent_severe': [0],
'damage_type_met_tear': [0], 
'damage_type_mis_lamp': [0], 
'damage_type_mis_lost': [0],
'damage_type_mis_punct': [0]
})


# Adjust case DataFrame based on selected_workshop_qual
if selected_workshop_qual == 'High':
    case['workshop_quality_Low'] = [0]
    case['workshop_quality_Medium'] = [0]
elif selected_workshop_qual == 'Medium':
    case['workshop_quality_Low'] = [0]
    case['workshop_quality_Medium'] = [1]
elif selected_workshop_qual == 'Low':
    case['workshop_quality_Low'] = [1]
    case['workshop_quality_Medium'] = [0]
    
case

,const,brand_Volkswagen,model_Corolla,model_Golf,model_Polo,model_Tiguan,model_Yaris,veh_age_range_Newer,veh_age_range_Old,workshop_quality_Low,workshop_quality_Medium,counties_group2,counties_group3,damage_type_met_dent_medium,damage_type_met_dent_minor,damage_type_met_dent_severe,damage_type_met_tear,damage_type_mis_lamp,damage_type_mis_lost,damage_type_mis_punct
0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0


In [71]:
damage_types = [results[0].names[int(cls)] for cls in results[0].boxes.cpu().numpy().cls]

# Function to generate the complete_case DataFrame
def generate_complete_case(case, damage_types):
    complete_case = pd.DataFrame()
    
    for damage_type in damage_types:
        temp_case = case.copy()
        
        # Set the corresponding damage type to 1
        damage_column = f'damage_type_{damage_type}'
        if damage_column in temp_case.columns:
            temp_case[damage_column] = [1]
        
        # Append this row to the complete_case DataFrame
        complete_case = pd.concat([complete_case, temp_case], ignore_index=True)
    
    return complete_case

# Generate the complete_case DataFrame
complete_case = generate_complete_case(case, damage_types)

complete_case.T

,0,1
const,1,1
brand_Volkswagen,0,0
model_Corolla,0,0
model_Golf,0,0
model_Polo,0,0
model_Tiguan,0,0
model_Yaris,1,1
veh_age_range_Newer,1,1
veh_age_range_Old,0,0
workshop_quality_Low,0,0


In [72]:
cost_model.predict(complete_case)

array([     280.92,      475.33])